In [1]:
import os
import pandas as pd
import glob
import dbfread
from sklearn import preprocessing

C:\Users\GeorgiyLebedev\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Функция для преобразования DBF в XLSX
def convert_dbf_to_xlsx(input_dbf, output_xlsx):
    # Читаем файл DBF
    table = dbfread.DBF(input_dbf, encoding='cp1251')  # Используйте подходящую кодировку
    records = []

    for record in table:
        # Добавляем запись в список
        records.append({field: (str(value) if value is not None else '') for field, value in record.items()})

    df = pd.DataFrame(records)

    # Записываем в Excel
    df.to_excel(output_xlsx, sheet_name='Sheet1', index=False)

In [3]:
# Функция создания показателей после 03.2022
def calculate_bank_metrics_2022_2024(df):
    # Исходный DataFrame для конкретного банка
    bank_df = df.copy()
    
    # Активы
    ACT = (
        bank_df[
            (bank_df["NUM_SC"].isin([20, 301, 302, 304, 319, 32.1, 32.2, 324, 501, 502, 504, 505, 506, 507,
                                     512, 513, 515, 601, 602, 526, 604, 608, 303, 306, 603, 325, 509, 610,
                                     620, 621, 528, 609, 617, 619]))
            & (bank_df["A_P"] == 1)
        ]["IITG"].sum()
        + bank_df[
            (bank_df["NUM_SC"].astype(str).str.startswith("4"))
            & (bank_df["A_P"] == 1)
        ]["IITG"].sum()
        - bank_df[
            (bank_df["NUM_SC"].isin([32.1, 32.2, 324, 501, 502, 504, 505, 506, 507, 512, 513, 515, 601, 602, 604, 303,
                                     509, 528, 325, 610, 620, 621, 609, 619, 45.1, 45.2, 47.1, 446, 449, 450, 451, 454,
                                     458, 470, 442, 453, 447, 448, 477, 472, 469, 462, 444, 445, 465, 461, 468, 441,
                                     443, 463, 464, 466, 467]))
            & (bank_df["A_P"] == 2)
        ]["IITG"].sum()
    )
    
    # Баланс
    BAL = (
        bank_df[
            (bank_df["NUM_SC"].isin([102, 105, 106, 107, 108, 109, 707, 708, 706, 111, 114]))
            & (bank_df["A_P"] == 2)
        ]["IITG"].sum()
        - bank_df[
            (bank_df["NUM_SC"].isin([706, 114, 109, 707, 708, 106]))
            & (bank_df["A_P"] == 1)
        ]["IITG"].sum()
    )
    
    # Капитализация
    CAP = BAL / ACT if ACT != 0 else None      

    
    # Работающие активы
    R_ASS = (
        bank_df[
        (bank_df['NUM_SC'].isin([32.1, 32.2, 324, 501, 502, 504, 505, 506, 507, 
                                       512, 513, 515, 601, 602, 446, 447, 449, 450, 45.1, 453, 465, 466,
                                       468, 469, 47.1, 472, 445, 448, 451, 464, 467, 470, 454, 441, 442, 
                                       443, 444, 460, 461, 462, 463, 45.2, 403, 477,
                                       458, 526]))
        &(bank_df['A_P']==1)
        ]['IITG'].sum() -bank_df[
    (bank_df['NUM_SC'].isin([320, 321, 322, 323, 324, 32.1, 32.2, 501, 502, 504, 505, 506, 507, 512, 513, 
                             515, 601, 602, 45.1, 45.2, 47.1, 446, 449, 450, 451, 454, 
                            458, 470, 442, 453, 447, 448, 477, 472, 469, 462, 444, 445, 465, 461, 468, 441, 
                            443, 463, 464, 466, 467]))
            &(bank_df['A_P']==2)
        ]['IITG'].sum()
    )
    
    QA = R_ASS/ACT if ACT != 0 else None  
    
    # Кредиты
    CR = bank_df[
        (bank_df["NUM_SC"].isin([32.1, 32.2]))
        & (bank_df["A_P"] == 1)
    ]["IITG"].sum()
    
    
    # Показатель CO
    CO = CR / R_ASS if R_ASS != 0 else None  
        
    # Доходы
    EARN = bank_df[(bank_df['NUM_SC'].isin([108, 707, 708, 706]))&(bank_df['A_P']==2)]['IITG'].sum()- bank_df[
    (bank_df['NUM_SC'].isin([109, 707, 708, 706]))&(bank_df['A_P']==1)]['IITG'].sum() 
    
    # Показатель PR
    PR = EARN / R_ASS if R_ASS != 0 else None  
        
    # Ликвидность
    LIQ = bank_df[
        (bank_df["NUM_SC"].isin([20]))
        & (bank_df["A_P"] == 1)
    ]["IITG"].sum()
    
    C_LIQ = LIQ/ R_ASS if R_ASS != 0 else None

    return {
        "REGN": bank_df["REGN"].unique()[0],
        "Capital": CAP,
        "Assets Quality": QA,
        "Management": CO,
        "Earnings": PR,
        "Liquidity": C_LIQ,
    }

In [4]:
# Функция для подготвки файла к работе
def process_dataset(df):
    # Оставляем только нужные столбцы
    columns_to_keep = ['REGN', 'NUM_SC', 'A_P', 'IITG', 'DT']
    df = df[columns_to_keep].copy()
    
    # Преобразуем значения столбца NUM_SC в float
    df['NUM_SC'] = pd.to_numeric(df['NUM_SC'], errors='coerce')
    
    # Убираем строки, где преобразование не получилось
    df.dropna(subset=['NUM_SC'], inplace=True)
    
    return df

In [5]:
# Функция для чтения одного файла и добавления колонки 'period'
def read_file_and_add_period(file_path, period):
    df = pd.read_excel(file_path)
    df['period'] = period
    return df

In [ ]:
# Указываем путь к каталогу с файлами
input_folder = r'C:\Users\GeorgiyLebedev\Desktop\Python\КР\архивы с 2023'
output_folder = r'C:\Users\GeorgiyLebedev\Desktop\Python\КР\данные с 2023'

# Переводим все файлы из input_folder в output_folder
for filename in os.listdir(input_folder):
    if filename.endswith('B1.dbf'):
        input_filepath = os.path.join(input_folder, filename)
        output_filename = f'{filename[:-4]}.xlsx'  # Добавляем .xlsx к имени
        output_filepath = os.path.join(output_folder, output_filename)
        print(f'Конвертирую {input_filepath} в {output_filepath}')
        convert_dbf_to_xlsx(input_filepath, output_filepath)

In [6]:
# Указываем путь до каталога с файлами
folder_path = r'C:\Users\GeorgiyLebedev\Desktop\Python\КР\данные с 2023'  

# Получаем список всех файлов в указанном каталоге, соответствующих шаблону *B1.xlsx
file_paths = glob.glob(os.path.join(folder_path, '*B1.xlsx'))

dfs = []  # Список датафреймов
processed_dfs = []

# Чтение всех файлов и добавление периода
for file_path in file_paths:
    period = file_path.split('.')[0][-8:-2]  # Извлекаем период из названия файла
    dfs.append(read_file_and_add_period(file_path, period))

for dataset in dfs:
    processed_df = process_dataset(dataset)
    processed_dfs.append(processed_df)

# Объединяем все датафреймы в один
combined_df = pd.concat(processed_dfs, ignore_index=True)

In [7]:
# Теперь работаем с объединенным датафреймом
grouped = combined_df.groupby(['REGN', 'DT'])
metrics_list = []

for (regn, period), group in grouped:
    metrics = calculate_bank_metrics_2022_2024(group)
    if metrics:
        metrics.update({'REGN': regn, 'DT': period})
        metrics_list.append(metrics)

result_df = pd.DataFrame(metrics_list)

In [8]:
result_df

,REGN,Capital,Assets Quality,Management,Earnings,Liquidity,DT
0,1,0.230167,0.595809,0.321769,0.323652,0.025854,2023-06-01
1,1,0.242275,0.621071,0.317875,0.324647,0.029282,2023-07-01
2,1,0.216952,0.628448,0.405069,0.288369,0.028986,2023-08-01
3,1,0.210213,0.635991,0.442435,0.283470,0.032946,2023-09-01
4,1,0.226756,0.582070,0.347995,0.334910,0.038586,2023-10-01
...,...,...,...,...,...,...,...
6584,3548,0.056877,0.000000,NaN,NaN,NaN,2024-10-01
6585,3548,0.063871,0.000000,NaN,NaN,NaN,2024-11-01
6586,3548,0.068599,0.000000,NaN,NaN,NaN,2024-12-01
6587,3550,0.908406,0.000000,NaN,NaN,NaN,2024-11-01


In [9]:
# Создаем функцию для работы с данными до 03.2022
list_values = sorted(set(combined_df['NUM_SC']))
def calculate_bank_metrics_2020(df):
    # Исходный DataFrame для конкретного банка
    bank_df = df.copy()
    
    numbers_to_include = '{102П} + {105А} + {10601П} + {10611П} + {10602П} + {10603П} - {10605А} + {10609П} - '\
'{10610А} + {10622П} - {10623А} + {10628П} - {10629А} + {10612П} - {10613А} + {10619П} - {10620А} + {10624П} - '\
'{10625А} + {52801П} - {52802А} + {52803П} - {52804А} + {108П} - {109А} + {707П} - {707А} + {708П} - {708А} + '\
'{706П} - {706А} + {10701П} + {10614П} + {10621П} + {10627П} - {10626А} + {10630П} + {10631П} + {10632П} - '\
'{10633А} + {10634П} - {10635А}'
    numbers_to_include = list(numbers_to_include.split())
    numbers_to_include = [i.replace('{', '').replace('}', '') for i in numbers_to_include if i.startswith('{')]
    numbers_to_include_A=[i.replace('А', '') for i in numbers_to_include if 'А' in i]
    numbers_to_include_A = list(map(float, numbers_to_include_A))
    numbers_to_include_P=[i.replace('П', '') for i in numbers_to_include if 'П' in i]
    numbers_to_include_P = list(map(float, numbers_to_include_P))

    first_numbers_to_include_A = ['105', '109', '707', '708', '706']
    first_numbers_to_include_P = ['102', '108', '707', '708', '706']
    first_numbers_to_include_A = [i for i in first_numbers_to_include_A if i not in list_values]
    first_numbers_to_include_P = [i for i in first_numbers_to_include_P if i not in list_values]


    BAL = bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_P))&(bank_df['A_P']==2)]['IITG'].sum() + bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_P)) & (bank_df['A_P'] == 2)]['IITG'].sum(
    ) - bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_A))&(bank_df['A_P']==1)]['IITG'].sum() - bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_A)) & (bank_df['A_P'] == 1)]['IITG'].sum()
    
    numbers_to_include = '{102П} + {105А} + {10601П} + {10611П} + {10602П} + {10603П} - {10605А} + {10609П} - '\
'{10610А} + {10622П} - {10623А} + {10628П} - {10629А} + {10612П} - {10613А} + {10619П} - {10620А} + {10624П} - '\
'{10625А} + {52801П} - {52802А} + {52803П} - {52804А} + {108П} - {109А} + {707П} - {707А} + {708П} - {708А} + '\
'{706П} - {706А} + {10701П} + {10614П} + {10621П} + {10627П} - {10626А} + {10630П} + {10631П} + {10632П} - '\
'{10633А} + {10634П} - {10635А} + {20321П} + {30126П} - {30128А} + {30129П} + {30607П} - {30608А} + {30609П} + '\
'{32115П} - {32116А} + {32117П} + {32311П} - {32312А} + {32313П} + {32403П} - {32407А} + {32408П} + {45615П} - '\
'{45616А} + {45617П} + {45715П} - {45713А} + {45714П} + {45818П} - {45820А} + {45821П} + {47308П} - {47312А} + '\
'{47313П} + {47425П} - {47465А} + {47466П} + {60324П} - {60351А} + {60352П} + {31210П} + {31213П} + {31201П} + '\
'{31214П} + {31202П} + {31203П} + {31215П} + {31216П} + {31204П} + {31217П} + {31205П} + {31218П} + {31206П} + '\
'{31219П} + {31207П} + {31220П} + {31221П} + {31212П} + {32901П} + {31222П} + {31701П} + {31704П} + {31801П} + '\
'{31804П} + {30109П} + {30116П} + {30219П} + {30236П} + {30111П} + {30122П} + {30123П} + {30230П} + {30231П} + '\
'{30117П} + {30411П} + {30412П} + {30414П} + {30415П} + {31310П} + {31410П} + {31501П} + {31601П} + {31302П} + '\
'{31402П} + {31502П} + {31602П} + {31303П} + {31304П} + {31403П} + {31404П} + {31503П} + {31504П} + {31603П} + '\
'{31604П} + {31305П} + {31405П} + {31505П} + {31605П} + {31306П} + {31406П} + {31506П} + {31606П} + {31307П} + '\
'{31407П} + {31507П} + {31607П} + {31308П} + {31309П} + {31408П} + {31409П} + {31508П} + {31509П} + {31608П} + '\
'{31609П} + {31301П} + {31401П} + {20313П} + {20314П} + {31702П} + {31703П} + {31802П} + {31803П} + {20309П} + '\
'{20310П} + {30601П} + {30606П} + {408.1П} + {40802П} + {40825П} + {40804П} + {40805П} + {40809П} + {40811П} + '\
'{40814П} + {40815П} + {40818П} + {40819П} + {40807П} + {40821П} + {40822П} + {47401П} + {47418П} + {47601П} + '\
'{47602П} + {47606П} + {47607П} + {49999П} + {41402П} + {41502П} + {41602П} + {41702П} + {41802П} + {41902П} + '\
'{42002П} + {42102П} + {42202П} + {42502П} + {43102П} + {43202П} + {43302П} + {43402П} + {43502П} + {43602П} + '\
'{43702П} + {43802П} + {43902П} + {44002П} + {41403П} + {41404П} + {41503П} + {41504П} + {41603П} + {41604П} + '\
'{41703П} + {41704П} + {41803П} + {41804П} + {41903П} + {41904П} + {42003П} + {42004П} + {42103П} + {42104П} + '\
'{42203П} + {42204П} + {42503П} + {42504П} + {43103П} + {43104П} + {43203П} + {43204П} + {43303П} + {43304П} + '\
'{43403П} + {43404П} + {43503П} + {43504П} + {43603П} + {43604П} + {43703П} + {43704П} + {43803П} + {43804П} + '\
'{43903П} + {43904П} + {44003П} + {44004П} + {41405П} + {41505П} + {41605П} + {41705П} + {41805П} + {41905П} + '\
'{42005П} + {42105П} + {42205П} + {42505П} + {43105П} + {43205П} + {43305П} + {43405П} + {43505П} + {43605П} + '\
'{43705П} + {43805П} + {43905П} + {44005П} + {41406П} + {41407П} + {41506П} + {41507П} + {41606П} + {41607П} + '\
'{41706П} + {41707П} + {41806П} + {41807П} + {41906П} + {41907П} + {42006П} + {42007П} + {42106П} + {42107П} + '\
'{42206П} + {42207П} + {42506П} + {42507П} + {43106П} + {43107П} + {43206П} + {43207П} + {43306П} + {43307П} + '\
'{43406П} + {43407П} + {43506П} + {43507П} + {43606П} + {43607П} + {43706П} + {43707П} + {43806П} + {43807П} + '\
'{43906П} + {43907П} + {44006П} + {44007П} + {41401П} + {41501П} + {41601П} + {41701П} + {41801П} + {41901П} + '\
'{42001П} + {42101П} + {42201П} + {42501П} + {43101П} + {43201П} + {43301П} + {43401П} + {43501П} + {43601П} + '\
'{43701П} + {43801П} + {43901П} + {44001П} + {40101П} + {40105П} + {40106П} + {40107П} + {40108П} - {40109А} + '\
'{40110П} - {40111А} + {40116П} + {402П} + {403П} + {404П} + {40102П} + {41001П} + {41101П} + '\
'{41201П} + {41301П} + {42701П} + {42801П} + {42901П} + {43001П} + {41002П} + {41102П} + {41202П} + {41302П} + '\
'{42702П} + {42802П} + {42902П} + {43002П} + {41003П} + {41004П} + {41103П} + {41104П} + {41203П} + {41204П} + '\
'{41303П} + {41304П} + {42703П} + {42704П} + {42803П} + {42804П} + {42903П} + {42904П} + {43003П} + {43004П} + '\
'{41005П} + {41105П} + {41205П} + {41305П} + {42705П} + {42805П} + {42905П} + {43005П} + {41006П} + {41007П} + '\
'{41106П} + {41107П} + {41206П} + {41207П} + {41306П} + {41307П} + {42706П} + {42707П} + {42806П} + {42807П} + '\
'{42906П} + {42907П} + {43006П} + {43007П} + {40803П} + {40810П} + {40813П} + {40817П} + {40820П} + {40823П} + '\
'{40824П} + {40826П} + {47603П} + {47605П} + {42301П} + {42309П} + {42601П} + {42609П} + {42302П} + {42310П} + '\
'{42602П} + {42610П} + {42303П} + {42304П} + {42311П} + {42312П} + {42603П} + {42604П} + {42611П} + {42612П} + '\
'{42305П} + {42313П} + {42605П} + {42613П} + {42306П} + {42307П} + {42314П} + {42315П} + {42606П} + {42607П} + '\
'{42614П} + {42615П} + {30220П} + {30223П} + {30227П} + {42108П} + {42109П} + {42110П} + {42111П} + {42112П} + '\
'{42113П} + {42114П} + {52001П} + {52002П} + {52003П} + {52004П} + {52005П} + {52006П} + {52401П} + {52101П} + '\
'{52201П} + {52102П} + {52202П} + {52103П} + {52203П} + {52104П} + {52204П} + {52105П} + {52106П} + {52205П} + '\
'{52206П} + {52403П} + {52404П} + {52301П} + {52302П} + {52303П} + {52304П} + {52305П} + {52306П} + {52307П} + '\
'{52406П} + {47446П} - {47451А} + {47454П} - {47460А} + {52402П} + {52405П} + {52407П} + {52501П} + {47411П} + '\
'{47426П} + {47608П} + {47609П} + {303П} - {303А} + {30222П} - {30221А} + {47403П} + {47405П} + {47407П} + '\
'{47412П} + {47414П} + {47416П} + {47419П} + {47422П} + {603П} + {40907П} - {40908А} + {30232П} - {30233А} + '\
'{40312П} - {40313А} + {613П} + {30603П} + {30604П} + {60806П} + {61501П} + {47441П} - {47440А} + {47442П} - '\
'{47443А} + {47449П} + {47814П} - {47813А} + {47815П} - {47816А} + {47501П} - {47502А} + {47448П} - {47467А} + '\
'{47457П} - {47463А} + {47458П} - {47464А} + {52602П} + {47453П} - {47459А}'
    numbers_to_include = list(numbers_to_include.split())
    numbers_to_include = [i.replace('{', '').replace('}', '') for i in numbers_to_include if i.startswith('{')]
    numbers_to_include_A=[i.replace('А', '') for i in numbers_to_include if 'А' in i]
    numbers_to_include_A = list(map(float, numbers_to_include_A))
    numbers_to_include_P=[i.replace('П', '') for i in numbers_to_include if 'П' in i]
    numbers_to_include_P = list(map(float, numbers_to_include_P))

    first_numbers_to_include_A = ['105', '109', '707', '708', '706', '303']
    first_numbers_to_include_P = ['102', '108', '707', '708', '706', '405', '406', '407', '409', '402', '403',
                             '303', '603', '613']
    first_numbers_to_include_A = [i for i in first_numbers_to_include_A if i not in list_values]
    first_numbers_to_include_P = [i for i in first_numbers_to_include_P if i not in list_values]


# Фильтрация данных и суммирование
    ACT = bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_P))&(bank_df['A_P']==2)]['IITG'].sum() + bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_P)) & (bank_df['A_P'] == 2)]['IITG'].sum(
    ) - bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_A))&(bank_df['A_P']==1)]['IITG'].sum() - bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_A)) & (bank_df['A_P'] == 1)]['IITG'].sum()
    
    CAP = BAL / ACT if ACT != 0 else None
    
   
    numbers_to_include = '{31901А} + {32010А} + {32110А} + {32201А} + {32301А} + {32902А} + {31902А} + {32002А} + '\
'{32102А} + {32202А} + {32302А} + {31903А} + {31904А} + {32003А} + {32004А} + {32103А} + {32104А} + {32203А} + '\
'{32204А} + {32303А} + {32304А} + {31905А} + {32005А} + {32105А} + {32205А} + {32305А} + {31906А} + {32006А} + '\
'{32106А} + {32206А} + {32306А} + {31907А} + {32007А} + {32107А} + {32207А} + {32307А} + {31908А} + {31909А} + '\
'{32008А} + {32009А} + {32108А} + {32109А} + {32208А} + {32209А} + {32308А} + {32309А} + {32401А} + {32402А} + '\
'{32001А} + {32101А} + {44109А} + {44210А} + {44310А} + {44410А} + {46001А} + {46101А} + {46201А} + {46301А} + '\
'{44101А} + {44102А} + {44103А} + {44202А} + {44203А} + {44204А} + {44302А} + {44303А} + {44304А} + {44402А} + '\
'{44403А} + {44404А} + {46002А} + {46102А} + {46202А} + {46302А} + {44105А} + {44206А} + {44306А} + {44406А} + '\
'{46004А} + {46104А} + {46204А} + {46304А} + {44106А} + {44207А} + {44307А} + {44407А} + {46005А} + {46105А} + '\
'{46205А} + {46305А} + {44104А} + {44205А} + {44305А} + {44405А} + {46003А} + {46103А} + {46203А} + {46303А} + '\
'{44107А} + {44108А} + {44208А} + {44209А} + {44308А} + {44309А} + {44408А} + {44409А} + {46006А} + {46007А} + '\
'{46106А} + {46107А} + {46206А} + {46207А} + {46306А} + {46307А} + {45801А} + {45802А} + {45803А} + {45804А} + '\
'{44201А} + {44301А} + {44401А} + {44509А} + {44609А} + {44709А} + {44809А} + {44909А} + {45009А} + {45109А} + '\
'{45209А} + {45309А} + {45409А} + {45607А} + {46401А} + {46501А} + {46601А} + {46701А} + {46801А} + {46901А} + '\
'{47001А} + {47101А} + {47201А} + {47301А} + {44503А} + {44603А} + {44703А} + {44803А} + {44903А} + {45003А} + '\
'{45103А} + {45203А} + {45303А} + {45403А} + {45601А} + {46402А} + {46502А} + {46602А} + {46702А} + {46802А} + '\
'{46902А} + {47002А} + {47102А} + {47202А} + {47302А} + {44504А} + {44604А} + {44704А} + {44804А} + {44904А} + '\
'{45004А} + {45104А} + {45204А} + {45304А} + {45404А} + {45602А} + {46403А} + {46503А} + {46603А} + {46703А} + '\
'{46803А} + {46903А} + {47003А} + {47103А} + {47203А} + {47303А} + {44505А} + {44605А} + {44705А} + {44805А} + '\
'{44905А} + {45005А} + {45105А} + {45205А} + {45305А} + {45405А} + {45603А} + {46404А} + {46504А} + {46604А} + '\
'{46704А} + {46804А} + {46904А} + {47004А} + {47104А} + {47204А} + {47304А} + {44506А} + {44606А} + {44706А} + '\
'{44806А} + {44906А} + {45006А} + {45106А} + {45206А} + {45306А} + {45406А} + {45604А} + {46405А} + {46505А} + '\
'{46605А} + {46705А} + {46805А} + {46905А} + {47005А} + {47105А} + {47205А} + {47305А} + {44507А} + {44508А} + '\
'{44607А} + {44608А} + {44707А} + {44708А} + {44807А} + {44808А} + {44907А} + {44908А} + {45007А} + {45008А} + '\
'{45107А} + {45108А} + {45207А} + {45208А} + {45307А} + {45308А} + {45407А} + {45408А} + {45605А} + {45606А} + '\
'{46406А} + {46407А} + {46506А} + {46507А} + {46606А} + {46607А} + {46706А} + {46707А} + {46806А} + {46807А} + '\
'{46906А} + {46907А} + {47006А} + {47007А} + {47106А} + {47107А} + {47206А} + {47207А} + {47306А} + {47307А} + '\
'{45805А} + {45806А} + {45807А} + {45808А} + {45809А} + {45810А} + {45811А} + {45812А} + {45813А} + {45814А} + '\
'{45816А} + {44501А} + {44601А} + {44701А} + {44801А} + {44901А} + {45001А} + {45101А} + {45201А} + {45301А} + '\
'{45401А} + {45608А} + {45410А} + {45508А} + {45707А} + {45502А} + {45701А} + {45503А} + {45702А} + {45504А} + '\
'{45703А} + {45505А} + {45704А} + {45506А} + {45507А} + {45705А} + {45706А} + {45815А} + {45817А} + {45509А} + '\
'{45708А} + {45510А} + {45709А} + {51213А} + {51313А} + {51513А} + {51216А} + {51316А} + {51516А} + {51214А} + '\
'{51217А} + {51314А} + {51317А} + {51514А} + {51517А} + {51211А} + {51212А} + {51311А} + {51312А} + {51511А} + '\
'{51512А} + {51215А} + {51315А} + {51515А} + {47410А} + {47431А} + {20311А} + {20312А} + {20317А} + {20318А} + '\
'{40111А} - {40110П} + {40308А} + {40310А} + {47402А} + {60315А} + {47447А} - {47452П} + {47701А} + {47801A} + '\
'{47802A} + {47803A} + {47807A} - {47808П} + {50104А} + {50105А} + {50108А} + {50106А} + {50109А} + {50107А} + '\
'{50110А} + {50113А} + {50115А} + {50116А} + {50118А} + {50121А} - {50120П} + {50140А} - {50141П} + {50605А} + '\
'{50607А} + {50606А} + {50608А} + {50618А} + {50621А} - {50620П} + {47456А} - {47462П} + {47809А} - {47810П} + '\
'{50670А} - {50671П} + {50401А} + {50402А} + {50405А} + {50403А} + {50406А} + {50404А} + {50407А} + {50408А} + '\
'{50418А} + {50428А} - {50429П} + {50505А} + {50205А} + {50206А} + {50209А} + {50207А} + {50210А} + {50208А} + '\
'{50211А} + {50214А} + {50218А} + {50221А} - {50220П} + {50264А} - {50265П} + {508А} - {50809П} + {50705А} + '\
'{50707А} + {50706А} + {50708А} + {50709А} + {50718А} + {50721А} - {50720П} + {47455А} - {47461П} +{47811А} - '\
'{47812П} + {50770А} - {50771П} + {60101А} + {60103А} + {60102А} + {60104А} + {60201А} + {60203А} + {60202А} + '\
'{60204А} + {60106А} + {60121А} - {60120П} + {60221А} - {60220П} + {52601А}'
    numbers_to_include = list(numbers_to_include.split())
    numbers_to_include = [i.replace('{', '').replace('}', '') for i in numbers_to_include if i.startswith('{')]
    numbers_to_include_A=[i.replace('А', '') for i in numbers_to_include if 'А' in i]
    numbers_to_include_A = list(map(float, numbers_to_include_A))
    numbers_to_include_P=[i.replace('П', '') for i in numbers_to_include if 'П' in i]
    numbers_to_include_P = list(map(float, numbers_to_include_P))

    first_numbers_to_include_A = ['508']
    first_numbers_to_include_P = []
    first_numbers_to_include_A = [i for i in first_numbers_to_include_A if i not in list_values]
    first_numbers_to_include_P = [i for i in first_numbers_to_include_P if i not in list_values]


    # Фильтрация данных и суммирование
    R_ASS = bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_A))&(bank_df['A_P']==1)]['IITG'].sum() + bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_A)) & (bank_df['A_P'] == 1)]['IITG'].sum(
    ) - bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_P))&(
    bank_df['A_P']==2)]['IITG'].sum() - bank_df[(bank_df['NUM_SC'].astype(str).str[:3].isin(
    first_numbers_to_include_P)) & (bank_df['A_P'] == 2)]['IITG'].sum()
    
    QA = R_ASS/ACT if ACT != 0 else None 

    
    numbers_to_include = '{31901А} + {32010А} + {32110А} + {32201А} + {32301А} + {32902А} + {31902А} + {32002А} + '\
'{32102А} + {32202А} + {32302А} + {31903А} + {31904А} + {32003А} + {32004А} + {32103А} + {32104А} + {32203А} + '\
'{32204А} + {32303А} + {32304А} + {31905А} + {32005А} + {32105А} + {32205А} + {32305А} + {31906А} + {32006А} + '\
'{32106А} + {32206А} + {32306А} + {31907А} + {32007А} + {32107А} + {32207А} + {32307А} + {31908А} + {31909А} + '\
'{32008А} + {32009А} + {32108А} + {32109А} + {32208А} + {32209А} + {32308А} + {32309А} + {32401А} + {32402А} + '\
'{32001А} + {32101А} + {44109А} + {44210А} + {44310А} + {44410А} + {46001А} + {46101А} + {46201А} + {46301А} + '\
'{44101А} + {44102А} + {44103А} + {44202А} + {44203А} + {44204А} + {44302А} + {44303А} + {44304А} + {44402А} + '\
'{44403А} + {44404А} + {46002А} + {46102А} + {46202А} + {46302А} + {44105А} + {44206А} + {44306А} + {44406А} + '\
'{46004А} + {46104А} + {46204А} + {46304А} + {44106А} + {44207А} + {44307А} + {44407А} + {46005А} + {46105А} + '\
'{46205А} + {46305А} + {44104А} + {44205А} + {44305А} + {44405А} + {46003А} + {46103А} + {46203А} + {46303А} + '\
'{44107А} + {44108А} + {44208А} + {44209А} + {44308А} + {44309А} + {44408А} + {44409А} + {46006А} + {46007А} + '\
'{46106А} + {46107А} + {46206А} + {46207А} + {46306А} + {46307А} + {45801А} + {45802А} + {45803А} + {45804А} + '\
'{44201А} + {44301А} + {44401А} + {44509А} + {44609А} + {44709А} + {44809А} + {44909А} + {45009А} + {45109А} + '\
'{45209А} + {45309А} + {45409А} + {45607А} + {46401А} + {46501А} + {46601А} + {46701А} + {46801А} + {46901А} + '\
'{47001А} + {47101А} + {47201А} + {47301А} + {44503А} + {44603А} + {44703А} + {44803А} + {44903А} + {45003А} + '\
'{45103А} + {45203А} + {45303А} + {45403А} + {45601А} + {46402А} + {46502А} + {46602А} + {46702А} + {46802А} + '\
'{46902А} + {47002А} + {47102А} + {47202А} + {47302А} + {44504А} + {44604А} + {44704А} + {44804А} + {44904А} + '\
'{45004А} + {45104А} + {45204А} + {45304А} + {45404А} + {45602А} + {46403А} + {46503А} + {46603А} + {46703А} + '\
'{46803А} + {46903А} + {47003А} + {47103А} + {47203А} + {47303А} + {44505А} + {44605А} + {44705А} + {44805А} + '\
'{44905А} + {45005А} + {45105А} + {45205А} + {45305А} + {45405А} + {45603А} + {46404А} + {46504А} + {46604А} + '\
'{46704А} + {46804А} + {46904А} + {47004А} + {47104А} + {47204А} + {47304А} + {44506А} + {44606А} + {44706А} + '\
'{44806А} + {44906А} + {45006А} + {45106А} + {45206А} + {45306А} + {45406А} + {45604А} + {46405А} + {46505А} + '\
'{46605А} + {46705А} + {46805А} + {46905А} + {47005А} + {47105А} + {47205А} + {47305А} + {44507А} + {44508А} + '\
'{44607А} + {44608А} + {44707А} + {44708А} + {44807А} + {44808А} + {44907А} + {44908А} + {45007А} + {45008А} + '\
'{45107А} + {45108А} + {45207А} + {45208А} + {45307А} + {45308А} + {45407А} + {45408А} + {45605А} + {45606А} + '\
'{46406А} + {46407А} + {46506А} + {46507А} + {46606А} + {46607А} + {46706А} + {46707А} + {46806А} + {46807А} + '\
'{46906А} + {46907А} + {47006А} + {47007А} + {47106А} + {47107А} + {47206А} + {47207А} + {47306А} + {47307А} + '\
'{45805А} + {45806А} + {45807А} + {45808А} + {45809А} + {45810А} + {45811А} + {45812А} + {45813А} + {45814А} + '\
'{45816А} + {44501А} + {44601А} + {44701А} + {44801А} + {44901А} + {45001А} + {45101А} + {45201А} + {45301А} + '\
'{45401А} + {45608А} + {45410А} + {45508А} + {45707А} + {45502А} + {45701А} + {45503А} + {45702А} + {45504А} + '\
'{45703А} + {45505А} + {45704А} + {45506А} + {45507А} + {45705А} + {45706А} + {45815А} + {45817А} + {45509А} + '\
'{45708А} + {45510А} + {45709А} + {51213А} + {51313А} + {51513А} + {51216А} + {51316А} + {51516А} + {51214А} + '\
'{51217А} + {51314А} + {51317А} + {51514А} + {51517А} + {51211А} + {51212А} + {51311А} + {51312А} + {51511А} + '\
'{51512А} + {51215А} + {51315А} + {51515А} + {47410А} + {47431А} + {20311А} + {20312А} + {20317А} + {20318А} + '\
'{40111А} - {40110П} + {40308А} + {40310А} + {47402А} + {60315А} + {47447А} - {47452П}' 
    numbers_to_include = list(numbers_to_include.split())
    numbers_to_include = [i.replace('{', '').replace('}', '') for i in numbers_to_include if i.startswith('{')]
    numbers_to_include_A=[i.replace('А', '') for i in numbers_to_include if 'А' in i]
    numbers_to_include_A = list(map(float, numbers_to_include_A))
    numbers_to_include_P=[i.replace('П', '') for i in numbers_to_include if 'П' in i]
    numbers_to_include_P = list(map(float, numbers_to_include_P))

    first_numbers_to_include_A = ['508']
    first_numbers_to_include_P = []
    first_numbers_to_include_A = [i for i in first_numbers_to_include_A if i not in list_values]
    first_numbers_to_include_P = [i for i in first_numbers_to_include_P if i not in list_values]

    # Фильтрация данных и суммирование
    CR = bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_A))&(bank_df['A_P']==1)]['IITG'].sum() + bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_A)) & (bank_df['A_P'] == 1)]['IITG'].sum(
    ) - bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_P))&(
    bank_df['A_P']==2)]['IITG'].sum() - bank_df[(bank_df['NUM_SC'].astype(str).str[:3].isin(
    first_numbers_to_include_P)) & (bank_df['A_P'] == 2)]['IITG'].sum()
        
    CO = CR / R_ASS if R_ASS != 0 else None     
    
    numbers_to_include = ''
    numbers_to_include = list(numbers_to_include.split())
    numbers_to_include = [i.replace('{', '').replace('}', '') for i in numbers_to_include if i.startswith('{')]
    numbers_to_include_A=[i.replace('А', '') for i in numbers_to_include if 'А' in i]
    numbers_to_include_A = list(map(int, numbers_to_include_A))
    numbers_to_include_P=[i.replace('П', '') for i in numbers_to_include if 'П' in i]
    numbers_to_include_P = list(map(int, numbers_to_include_P))

    first_numbers_to_include_A = ['109', '707', '708', '706']
    first_numbers_to_include_P = ['108', '707', '708', '706']
    first_numbers_to_include_A = [i for i in first_numbers_to_include_A if i not in list_values]
    first_numbers_to_include_P = [i for i in first_numbers_to_include_P if i not in list_values]


    # Фильтрация данных и суммирование
    EARN = bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_P))&(bank_df['A_P']==2)]['IITG'].sum() + bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_P)) & (bank_df['A_P'] == 2)]['IITG'].sum(
    )  - bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_A))&(
    bank_df['A_P']==1)]['IITG'].sum() - bank_df[(bank_df['NUM_SC'].astype(str).str[:3].isin(
    first_numbers_to_include_A)) & (bank_df['A_P'] == 1)]['IITG'].sum() 
    
    PR = EARN / R_ASS if R_ASS != 0 else None 
    
    numbers_to_include = '{20202А} + {20203А} + {20208А} + {20209А} + {20210А} + {20302А} + {20303А} + {20305А} + '\
'{20999А} + {30417А} + {30419А} + {30208А} + {30224А} + {30102А} + {30104А} + {30106А} + {30125А} + {30210А} + '\
'{30228А} + {30235А} + {30110А} + {30118А} + {30215А} + {30119А} + {30114А} + {30413А} + {304164} + {30418A} + '\
'{30424A} + {30425A} + {30427A} + {30428А} + {30429П} - {30420П} - {30421П} - {30422П} - {30423П} + {304.1A} + '\
'{20315А} + {20316А}'
    numbers_to_include = list(numbers_to_include.split())
    numbers_to_include = [i.replace('{', '').replace('}', '') for i in numbers_to_include if i.startswith('{')]
    numbers_to_include_A=[i.replace('А', '') for i in numbers_to_include if 'А' in i]
    numbers_to_include_A = list(map(float, numbers_to_include_A))
    numbers_to_include_P=[i.replace('П', '') for i in numbers_to_include if 'П' in i]
    numbers_to_include_P = list(map(float, numbers_to_include_P))

    first_numbers_to_include_A = ['204']
    first_numbers_to_include_P = []
    first_numbers_to_include_A = [i for i in first_numbers_to_include_A if i not in list_values]
    first_numbers_to_include_P = [i for i in first_numbers_to_include_P if i not in list_values]


    # Фильтрация данных и суммирование
    LIQ = bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_A))&(bank_df['A_P']==1)]['IITG'].sum() + bank_df[
    (bank_df['NUM_SC'].astype(str).str[:3].isin(first_numbers_to_include_A)) & (bank_df['A_P'] == 1)]['IITG'].sum(
    ) - bank_df[(bank_df['NUM_SC'].isin(numbers_to_include_P))&(
    bank_df['A_P']==2)]['IITG'].sum() - bank_df[(bank_df['NUM_SC'].astype(str).str[:3].isin(
    first_numbers_to_include_P)) & (bank_df['A_P'] == 2)]['IITG'].sum()
    
    C_LIQ = LIQ/ R_ASS if R_ASS != 0 else None
    
    
    return {
        "REGN": bank_df["REGN"].unique()[0],
        "Capital": CAP,
        "Assets Quality": QA,
        "Management": CO,
        "Earnings": PR,
        "Liquidity": C_LIQ,
    }

In [ ]:
# Указываем путь к каталогу с файлами
input_folder = r'C:\Users\GeorgiyLebedev\Desktop\Python\КР\архивы с 2020 до 2022'
output_folder = r'C:\Users\GeorgiyLebedev\Desktop\Python\КР\данные с 2020 до 2022'

# Переводим все файлы из input_folder в output_folder
for filename in os.listdir(input_folder):
    if filename.endswith('B1.DBF'):
        input_filepath = os.path.join(input_folder, filename)
        output_filename = f'{filename[:-4]}.xlsx'  # Добавляем .xlsx к имени
        output_filepath = os.path.join(output_folder, output_filename)
        print(f'Конвертирую {input_filepath} в {output_filepath}')
        convert_dbf_to_xlsx(input_filepath, output_filepath)

In [10]:
# Указываем путь до каталога с файлами
folder_path = r'C:\Users\GeorgiyLebedev\Desktop\Python\КР\данные с 2020 до 2022'

# Получаем список всех файлов в указанном каталоге, соответствующих шаблону *B1.xlsx
file_paths = glob.glob(os.path.join(folder_path, '*B1.xlsx'))

dfs = []  
processed_dfs = []

# Чтение всех файлов и добавление периода
for file_path in file_paths:
    period = file_path.split('.')[0][-8:-2]  # Извлекаем период из названия файла
    dfs.append(read_file_and_add_period(file_path, period))

for dataset in dfs:
    processed_df = process_dataset(dataset)
    processed_dfs.append(processed_df)

# Объединяем все датафреймы в один
combined_df = pd.concat(processed_dfs, ignore_index=True)

In [11]:
# Теперь работаем с объединенным датафреймом
grouped = combined_df.groupby(['REGN', 'DT'])
metrics_list = []

for (regn, period), group in grouped:
    metrics = calculate_bank_metrics_2020(group)
    if metrics:
        metrics.update({'REGN': regn, 'DT': period})
        metrics_list.append(metrics)

result_df_2020_2022 = pd.DataFrame(metrics_list)

In [12]:
result_df_2020_2022

,REGN,Capital,Assets Quality,Management,Earnings,Liquidity,DT
0,1,0.163842,0.922667,0.841655,0.129868,0.043859,2020-01-01
1,1,0.158347,0.877895,0.836208,0.135343,0.101964,2020-02-01
2,1,0.138251,0.897450,0.869713,0.117077,0.071777,2020-03-01
3,1,0.134778,0.884755,0.853008,0.114038,0.088960,2020-04-01
4,1,0.133335,0.880895,0.858602,0.114726,0.087442,2020-05-01
...,...,...,...,...,...,...,...
10184,3540,0.894696,3.405857,0.989988,0.038142,-0.777984,2021-12-01
10185,3540,0.718968,3.055769,0.990931,0.031881,-0.727309,2022-01-01
10186,3540,0.841215,3.431565,0.990519,0.032512,-0.768112,2022-02-01
10187,3541,0.996662,0.000000,NaN,NaN,NaN,2022-01-01


In [13]:
# Объединяем результаты обоих периодов и сортируем по регистрационному коду и периоду
combined_results = pd.concat([result_df_2020_2022, result_df], ignore_index=True)
combined_results = combined_results.sort_values(by=['REGN', 'DT'])
combined_results['DT'] = pd.to_datetime(combined_results['DT'])

In [14]:
combined_results

,REGN,Capital,Assets Quality,Management,Earnings,Liquidity,DT
0,1,0.163842,0.922667,0.841655,0.129868,0.043859,2020-01-01
1,1,0.158347,0.877895,0.836208,0.135343,0.101964,2020-02-01
2,1,0.138251,0.897450,0.869713,0.117077,0.071777,2020-03-01
3,1,0.134778,0.884755,0.853008,0.114038,0.088960,2020-04-01
4,1,0.133335,0.880895,0.858602,0.114726,0.087442,2020-05-01
...,...,...,...,...,...,...,...
16773,3548,0.056877,0.000000,NaN,NaN,NaN,2024-10-01
16774,3548,0.063871,0.000000,NaN,NaN,NaN,2024-11-01
16775,3548,0.068599,0.000000,NaN,NaN,NaN,2024-12-01
16776,3550,0.908406,0.000000,NaN,NaN,NaN,2024-11-01


In [15]:
# Убираем строки с пропущенными данными
combined_results_clean = combined_results.dropna()

In [16]:
combined_results_clean

,REGN,Capital,Assets Quality,Management,Earnings,Liquidity,DT
0,1,0.163842,0.922667,0.841655,0.129868,0.043859,2020-01-01
1,1,0.158347,0.877895,0.836208,0.135343,0.101964,2020-02-01
2,1,0.138251,0.897450,0.869713,0.117077,0.071777,2020-03-01
3,1,0.134778,0.884755,0.853008,0.114038,0.088960,2020-04-01
4,1,0.133335,0.880895,0.858602,0.114726,0.087442,2020-05-01
...,...,...,...,...,...,...,...
16755,3547,0.981229,0.948155,1.000096,-0.006884,0.000000,2024-02-01
16756,3547,0.972796,0.917949,1.000143,-0.004233,0.000000,2024-03-01
16762,3547,0.853369,0.000002,1.000000,-10710.000000,0.000000,2024-09-01
16763,3547,0.850948,0.000005,1.000000,-4792.333333,0.000000,2024-10-01


In [17]:
# Сохраняем результат в EXCEL-файл
combined_results_clean.to_excel("bank_metrics.xlsx", index=False)

In [18]:
# Функция для удаления выбросов
def remove_outliers(df, exclude_columns):
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        if column not in exclude_columns:
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Удаляем выбросы
            df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    return df

In [19]:
combined_results_final = remove_outliers(combined_results_clean, exclude_columns=['REGN', 'DT'])

In [20]:
columns_to_normalize = combined_results_final.columns.difference(['REGN', 'DT'])

In [21]:
# Нормализуем данные
normalizer = preprocessing.MinMaxScaler()
combined_results_final[columns_to_normalize] = normalizer.fit_transform(combined_results_final[columns_to_normalize])

In [22]:
combined_results_final

,REGN,Capital,Assets Quality,Management,Earnings,Liquidity,DT
0,1,0.421474,0.693402,0.456604,0.472663,0.319901,2020-01-01
1,1,0.414613,0.663190,0.453649,0.479047,0.495556,2020-02-01
2,1,0.389523,0.676386,0.471826,0.457745,0.404301,2020-03-01
3,1,0.385186,0.667818,0.462763,0.454201,0.456244,2020-04-01
4,1,0.383385,0.665214,0.465798,0.455002,0.451657,2020-05-01
...,...,...,...,...,...,...,...
16731,3545,0.264495,0.386697,0.493247,0.387271,0.187315,2024-08-01
16732,3545,0.271550,0.438611,0.500422,0.390488,0.187315,2024-09-01
16733,3545,0.277080,0.430665,0.499071,0.401493,0.187315,2024-10-01
16734,3545,0.280067,0.477299,0.505222,0.397901,0.187315,2024-11-01


In [23]:
# Сохраняем результат в EXCEL-файл
combined_results_final.to_excel("bank_metrics_filtered.xlsx", index=False)

In [24]:

# Убираем те банки, чьи данные полностью отсутствуют в одном из периоде

# Определение временных интервалов
pre_gap_start = pd.to_datetime('2020-01-01')
pre_gap_end = pd.to_datetime('2022-02-01')
post_gap_start = pd.to_datetime('2023-06-01')
post_gap_end = pd.to_datetime('2024-12-01')

# Банки, которые есть хотя бы раз ДО пропуска
pre_gap_banks = combined_results_final[
    (combined_results_final['DT'] >= pre_gap_start) & 
    (combined_results_final['DT'] <= pre_gap_end)
]['REGN'].unique()

# Банки, которые есть хотя бы раз ПОСЛЕ пропуска
post_gap_banks = combined_results_final[
    (combined_results_final['DT'] >= post_gap_start) & 
    (combined_results_final['DT'] <= post_gap_end)
]['REGN'].unique()

# Банки, присутствующие в ОБОИХ периодах
valid_banks = set(pre_gap_banks) & set(post_gap_banks)

# Фильтрация исходного датасета
filtered_data = combined_results_final[combined_results_final['REGN'].isin(valid_banks)]

In [25]:
filtered_data

,REGN,Capital,Assets Quality,Management,Earnings,Liquidity,DT
0,1,0.421474,0.693402,0.456604,0.472663,0.319901,2020-01-01
1,1,0.414613,0.663190,0.453649,0.479047,0.495556,2020-02-01
2,1,0.389523,0.676386,0.471826,0.457745,0.404301,2020-03-01
3,1,0.385186,0.667818,0.462763,0.454201,0.456244,2020-04-01
4,1,0.383385,0.665214,0.465798,0.455002,0.451657,2020-05-01
...,...,...,...,...,...,...,...
16551,3531,0.775517,0.519180,0.004657,0.961423,0.231269,2023-07-01
16552,3531,0.789288,0.564691,0.011219,0.915296,0.261502,2023-08-01
16554,3531,0.763433,0.498038,0.005882,0.971768,0.338245,2023-10-01
16555,3531,0.731486,0.468218,0.029798,0.979422,0.410610,2023-11-01


In [26]:
# Сохраняем результат в EXCEL-файл
filtered_data.to_excel("bank_metrics_filtered_final.xlsx", index=False)

In [30]:
filtered_data['REGN'].nunique()

266

In [ ]:
df_banks['pattern_fix'].value_counts()